In [233]:
# Dependencies
import pandas as pd
import numpy as np
import os
import csv

from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base

In [234]:
# Save path to data set in a variable
data_file_cm = "clean_hawaii_measurements.csv"
data_file_cs = "clean_hawaii_stations.csv"

In [235]:
# Reading CSV file and displaying data as DataFrame
data_frame_cm = pd.read_csv(data_file_cm)
data_frame_cm.tail()

,station,date,prcp,tobs
18098,USC00516128,2017-08-17,0.13,72
18099,USC00516128,2017-08-19,0.09,71
18100,USC00516128,2017-08-21,0.56,76
18101,USC00516128,2017-08-22,0.50,76
18102,USC00516128,2017-08-23,0.45,76


In [236]:
# Reading CSV file and displaying data as DataFrame
data_frame_cs = pd.read_csv(data_file_cs)
data_frame_cs

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [237]:
# Reading Columns of DataFrame
data_frame_cs.columns

Index(['station', 'name', 'latitude', 'longitude', 'elevation'], dtype='object')

In [238]:
# To ignore warnings that gets triggered while executing same class more than once
import warnings
from sqlalchemy import exc as sa_exc

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=sa_exc.SAWarning)

In [239]:
# Define 'Measurement' table
class Measurement(Base):
    __tablename__ = 'measure'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    #index = Column(Integer)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

In [240]:
# Define 'Station' table
class Station(Base):
    __tablename__ = 'station'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    #index = Column(Integer)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [241]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'measure': Table('measure', MetaData(bind=None), Column('id', Integer(), table=<measure>, primary_key=True, nullable=False), Column('station', String(), table=<measure>), Column('date', String(), table=<measure>), Column('prcp', Float(), table=<measure>), Column('tobs', Integer(), table=<measure>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('station', String(), table=<station>), Column('name', String(), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)})

In [242]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

In [243]:
# Create our database engine
engine = create_engine('sqlite:///hawaii.sqlite')

# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [244]:
# To drop created tables
#Base.metadata.drop_all(engine)

#### Defining Reflection

In [245]:
Base_Reflect = automap_base()
Base_Reflect.prepare(engine, reflect=True)

In [246]:
Base_Reflect.classes.keys()

['measure', 'station']

In [247]:
# Loading data from Pandas 'DataFrame' to 'Sqlite' DB
Station_sqlite = data_frame_cs.to_sql(name='station', con=engine, if_exists='append', index=False)
Measurement_sqlite = data_frame_cm.to_sql(name='measure', con=engine, if_exists='append', index=False)

In [248]:
# Save reference to the table
Station = Base_Reflect.classes.station
Measurement = Base_Reflect.classes.measure

In [251]:
Station, Measurement

(sqlalchemy.ext.automap.station, sqlalchemy.ext.automap.measure)